In [1]:
import polars as pl
import os

ruta = r"C:\Users\etorres.DERCOPARTS\OneDrive - DERCO CHILE REPUESTOS SpA\Escritorio\SO\2022"

archivos_excel = [os.path.join(ruta, archivo) for archivo in os.listdir(ruta) if archivo.endswith('.xlsx')]

columnas_por_archivo = []
dataframes = []

for archivo in archivos_excel:
    df = pl.read_excel(archivo)
    dataframes.append(df)
    columnas_por_archivo.append(set(df.columns))

iguales = all(columnas == columnas_por_archivo[0] for columnas in columnas_por_archivo)

df_combinado = pl.concat(dataframes) if iguales else None

print(f"Se han combinado {len(archivos_excel)} archivos.")
print("Todas las columnas son iguales en todos los archivos." if iguales else "Las columnas no son iguales en todos los archivos.")

df_combinado
# Convertir Fe.contabilización al primer día del mes
df_combinado = df_combinado.with_columns(
    pl.col("Fe.contabilización")
    .cast(pl.Date)  # Asegurarse de que sea tipo fecha
    .dt.truncate("1mo")  # Ajustar al primer día del mes
)


# Agrupar por Centro, Material y Fe.contabilización, sumando Cantidad
df_agrupado = df_combinado.group_by(["Centro", "Material", "Fe.contabilización"]).agg(
    pl.col("Cantidad").sum().alias("Suma de Cantidad")
)
# Calcular Venta UMB como Suma de Cantidad * -1
df_agrupado = df_agrupado.with_columns(
    (pl.col("Suma de Cantidad") * -1).alias("Venta UMB")
)

df_agrupado


Se han combinado 25 archivos.
Todas las columnas son iguales en todos los archivos.


Centro,Material,Fe.contabilización,Suma de Cantidad,Venta UMB
str,str,date,i64,i64
"""0432""","""110185""",2022-05-01,-1,1
"""0450""","""1071296""",2022-12-01,-1,1
"""0484""","""160103""",2022-02-01,-1,1
"""0465""","""121841""",2022-08-01,-2,2
"""0432""","""121483""",2022-09-01,-1,1
…,…,…,…,…
"""0481""","""121009""",2022-10-01,-1,1
"""0432""","""106839""",2022-05-01,-2,2
"""0490""","""130674""",2022-11-01,-82,82


In [2]:
import polars as pl

# Ruta donde se guardará el archivo Parquet
output_path = r"C:\Users\etorres.DERCOPARTS\OneDrive - DERCO CHILE REPUESTOS SpA\Escritorio\SO\df_agrupado.parquet"

# Suponiendo que df_agrupado es tu DataFrame de Polars
# Guarda el DataFrame en formato Parquet
df_agrupado.write_parquet(output_path)

print(f"Archivo Parquet guardado en: {output_path}")


Archivo Parquet guardado en: C:\Users\etorres.DERCOPARTS\OneDrive - DERCO CHILE REPUESTOS SpA\Escritorio\SO\df_agrupado.parquet


In [ ]:
import polars as pl
import os
from concurrent.futures import ThreadPoolExecutor
# Ruta de la carpeta de archivos Excel
ruta = r"C:\Users\etorres.DERCOPARTS\OneDrive - DERCO CHILE REPUESTOS SpA\Escritorio\SO\2023-2024"

# Obtener la lista de archivos Excel en la ruta especificada
archivos_excel = [
    os.path.join(ruta, archivo)
    for archivo in os.listdir(ruta)
    if archivo.lower().endswith('.xlsx')
]

def leer_archivo(archivo):
    try:
        # Especificar las columnas a leer
        columnas_a_leer = ["Centro", "Material", "Fe.contabilización", "Cantidad"]
        df = pl.read_excel(archivo, columns=columnas_a_leer)
        
        # Verificar si todas las columnas están presentes
        columnas_faltantes = set(columnas_a_leer) - set(df.columns)
        if columnas_faltantes:
            print(f"El archivo {archivo} no contiene las columnas: {', '.join(columnas_faltantes)}")
            return None
        return df
    except Exception as e:
        print(f"Error al leer el archivo {archivo}: {e}")
        return None

# Leer todos los archivos
with ThreadPoolExecutor() as executor:
    dataframes = list(executor.map(leer_archivo, archivos_excel))

# Filtrar dataframes válidos
dataframes = [df for df in dataframes if df is not None]

if not dataframes:
    print("No se pudo leer ningún archivo correctamente.")
    exit()

# Asegurar que todas las columnas estén presentes en todos los DataFrames
columnas_unicas = set().union(*(df.columns for df in dataframes))

# Crear un esquema unificado para alinear tipos
esquema_unificado = {
    col: pl.Float64 if any(df[col].dtype == pl.Float64 for df in dataframes if col in df.columns) else
         pl.Int64 if all(df[col].dtype == pl.Int64 for df in dataframes if col in df.columns) else
         pl.Utf8
    for col in columnas_unicas
}

# Alinear todos los DataFrames al esquema unificado
dataframes = [
    df.select([
        pl.col(col).cast(esquema_unificado[col]) if col in df.columns else pl.lit(None, dtype=esquema_unificado[col]).alias(col)
        for col in columnas_unicas
    ])
    for df in dataframes
]

# Combinar los DataFrames
df_combinado = pl.concat(dataframes, how="vertical")

print(f"Se han combinado {len(dataframes)} archivos.")

# Convertir 'Fe.contabilización' al primer día del mes
if "Fe.contabilización" in df_combinado.columns:
    if df_combinado.schema["Fe.contabilización"] == pl.Utf8:
        df_combinado = df_combinado.with_columns(
            pl.col("Fe.contabilización")
            .str.to_date(strict=False)
            .dt.truncate("1mo")
            .alias("Fe.contabilización")
        )
    elif df_combinado.schema["Fe.contabilización"] == pl.Date:
        df_combinado = df_combinado.with_columns(
            pl.col("Fe.contabilización")
            .dt.truncate("1mo")
            .alias("Fe.contabilización")
        )
    else:
        # Intentar convertir a fecha si el tipo es diferente
        df_combinado = df_combinado.with_columns(
            pl.col("Fe.contabilización")
            .cast(pl.Date, strict=False)
            .dt.truncate("1mo")
            .alias("Fe.contabilización")
        )

# Asegurarse de que 'Cantidad' sea numérico si existe
if "Cantidad" in df_combinado.columns:
    df_combinado = df_combinado.with_columns(
        pl.col("Cantidad").cast(pl.Float64, strict=False)
    )

# Agrupar por 'Centro', 'Material', 'Fe.contabilización', sumando 'Cantidad'
if {"Centro", "Material", "Fe.contabilización", "Cantidad"}.issubset(df_combinado.columns):
    df_agrupado = df_combinado.group_by(["Centro", "Material", "Fe.contabilización"]).agg(
        pl.col("Cantidad").sum().alias("Suma de Cantidad")
    )

    # Calcular 'Venta UMB' como 'Suma de Cantidad' * -1
    df_agrupado = df_agrupado.with_columns(
        (pl.col("Suma de Cantidad") * -1).alias("Venta UMB")
    )

    print(df_agrupado)
else:
    print("No se puede agrupar porque faltan columnas necesarias.")


In [ ]:
import polars as pl

# Ruta donde se guardará el archivo Parquet
output_path = r"C:\Users\etorres.DERCOPARTS\OneDrive - DERCO CHILE REPUESTOS SpA\Escritorio\SO\df_agrupado2.parquet"

# Suponiendo que df_agrupado es tu DataFrame de Polars
# Guarda el DataFrame en formato Parquet
df_agrupado.write_parquet(output_path)

print(f"Archivo Parquet guardado en: {output_path}")

In [2]:
import pandas as pd
import numpy as np

file1 = "C:\\Users\\etorres.DERCOPARTS\\OneDrive - DERCO CHILE REPUESTOS SpA\\Escritorio\\SO\\2022.parquet"
file2 = "C:\\Users\\etorres.DERCOPARTS\\OneDrive - DERCO CHILE REPUESTOS SpA\\Escritorio\\SO\\2024-2023.parquet"

# Leer los archivos Parquet con pandas
data_2022 = pd.read_parquet(file1)
data_2023_2024 = pd.read_parquet(file2)

# Asegurar que los tipos de columnas sean consistentes
def convert_int_to_float(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('float64')
    return df

data_2022 = convert_int_to_float(data_2022)
data_2023_2024 = convert_int_to_float(data_2023_2024)

# Concatenar los DataFrames
df_agrupado = pd.concat([data_2022, data_2023_2024], ignore_index=True)

# Lista de centros de CL AGROPLANET
centros_cl_agroplanet = [
    "1624", "0480", "0482", "0483", "0484", "0485", "0486", "0487", "0488",
    "0489", "0490", "0491", "0492", "0493", "0494", "0496", "0498", "0499",
    "1627", "04A2", "04A3", "04A4", "04A5", "04A6", "04A7", "04A8", "04A9",
    "480", "482", "483", "484", "485", "486", "487", "488",
    "489", "490", "491", "492", "493", "494", "496", "498", "499",
    "4A2", "4A3", "4A4", "4A5", "4A6", "4A7", "4A8", "4A9"
]

# Asegurar que la columna 'Centro' es de tipo string
df_agrupado['Centro'] = df_agrupado['Centro'].astype(str)

# Crear la columna 'Canal' según la condición especificada
df_agrupado['Canal'] = np.where(
    df_agrupado['Centro'].isin(centros_cl_agroplanet),
    'CL AGROPLANET',
    'CL AUTOPLANET'
)


In [62]:
# Eliminar duplicados y ordenar por Centro
canal_por_centro = df_agrupado[['Centro', 'Canal']].drop_duplicates().sort_values(by='Centro')
# Ruta de exportación
ruta_exportacion = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\centros_y_canales_unicos.csv"

# Exportar a CSV delimitado por ';'
canal_por_centro.to_csv(ruta_exportacion, sep=';', index=False)

print(f"Archivo exportado correctamente a {ruta_exportacion}")



Archivo exportado correctamente a C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\centros_y_canales_unicos.csv


In [3]:
# Remove the first leading zero from 'Centro' column if present
df_agrupado['Centro'] = df_agrupado['Centro'].apply(lambda x: x[1:] if x.startswith('0') else x)
df_agrupado

,Centro,Material,Fe.contabilización,Suma de Cantidad,Venta UMB,Canal
0,432,110185,2022-05-01,-1.0,1.0,CL AUTOPLANET
1,450,1071296,2022-12-01,-1.0,1.0,CL AUTOPLANET
2,484,160103,2022-02-01,-1.0,1.0,CL AGROPLANET
3,465,121841,2022-08-01,-2.0,2.0,CL AUTOPLANET
4,432,121483,2022-09-01,-1.0,1.0,CL AUTOPLANET
...,...,...,...,...,...,...
4195445,479,1025289,2023-03-01,-4.0,4.0,CL AUTOPLANET
4195446,491,1090836,2024-01-01,-6.0,6.0,CL AGROPLANET
4195447,414,120995,2023-03-01,-6.0,6.0,CL AUTOPLANET
4195448,452,429946,2023-04-01,-3.0,3.0,CL AUTOPLANET


In [7]:
pivot_table.shape

(58890, 29)

In [4]:
# Crear la tabla pivot
pivot_table = df_agrupado.pivot_table(
    index=['Canal', 'Material'],
    columns='Fe.contabilización',
    values='Venta UMB',
    aggfunc='sum',
    fill_value=0
)

# Restablecer el índice para que se repitan los valores de 'Canal'
pivot_table = pivot_table.reset_index()

# Reemplazar valores negativos por 0
pivot_table = pivot_table.applymap(lambda x: 0 if isinstance(x, (int, float)) and x < 0 else x)

# Ruta de exportación
ruta_exportacion = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\pivot_table1.xlsx"

# Guardar como archivo Excel
pivot_table.to_excel(ruta_exportacion, index=False)

print(f"Archivo exportado correctamente a {ruta_exportacion}")


C:\Users\etorres.DERCOPARTS\AppData\Local\Temp\ipykernel_22152\3049664791.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_table = pivot_table.applymap(lambda x: 0 if isinstance(x, (int, float)) and x < 0 else x)


Archivo exportado correctamente a C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\pivot_table1.xlsx
